In [ ]:
from huggingface_hub import login

# Prompt the user for their Hugging Face token
huggingface_token = input("Enter your Hugging Face token: ")

# Log in to Hugging Face
login(token=huggingface_token)

print("Successfully logged in to Hugging Face!")


In [ ]:
from datasets import load_dataset

# Load the dataset from Hugging Face Hub
dataset_name = "akhilfau/physics_decontaminated_2"
dataset = load_dataset(dataset_name, split="train")  # Adjust the split if needed (e.g., "test" or "validation")

# Print a sample record
print("Sample record from the dataset:")
print(dataset[0])  # Prints the first record from the dataset


In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# Step 1: Load the Dataset
dataset = load_dataset("akhilfau/physics_decontaminated_2", split="train")

# Step 2: Load the Pretrained Model and Tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the padding token if not already set
tokenizer.pad_token = tokenizer.eos_token or tokenizer.bos_token or "[PAD]"

# Step 3: Configure LoRA with PEFT
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM",  # Task type for causal language modeling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters to confirm LoRA is applied
model.print_trainable_parameters()

# Step 4: Preprocess the Dataset
def preprocess_function(examples):
    # Concatenate the problem and solution for causal LM
    inputs = [f"Problem: {problem}\nSolution: {solution}" for problem, solution in zip(examples["message_1"], examples["message_2"])]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    
    # Labels are the same as input_ids for causal LM
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split into train and validation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Step 5: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./lora_fine_tuned_results",
    evaluation_strategy="epoch",
    logging_steps=100,
    save_steps=500,
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,  # Set to True if you want to push to Hugging Face Hub
    logging_dir="./logs",
)

# Step 6: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Step 7: Train the Model
trainer.train()

# Step 8: Save the Model and Tokenizer Locally
model.save_pretrained("./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics")
tokenizer.save_pretrained("./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics")

# Push to Hugging Face Hub
trainer.push_to_hub(commit_message="Fine-tuned smolLM2-135M with LoRA on camel-ai/physics")

from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="fine-tuned-smolLM2-135M-with-LoRA-on-camel-ai-physics",
    repo_id="akhilfau/fine-tuned-smolLM2-135M-with-LoRA-on-camel-ai-physics"
)



In [ ]:
# import torch

# # Clear cache
# torch.cuda.empty_cache()

# # Reset memory allocations and free up GPU memory
# torch.cuda.memory_summary(device=None, abbreviated=False)


In [ ]:
# import torch 
# for i in range(torch.cuda.device_count()):     
#     print(f"GPU {i}: {torch.cuda.get_device_name(i)}")